### Using as Instance Properties

Итак, давайте начнем изучать, как можно использовать дескрипторы для чтения и записи свойств экземпляра.

Сначала мы могли бы попробовать что-то вроде этого:

In [1]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return getattr(instance, 'stored_value', None)

По сути, мы собираемся использовать словарь экземпляров для сохранения значения под каким-то именем (символом) в нем - какое имя нам следует использовать? Это может быть проблемой, и мы к этому вернемся.

In [2]:
class Point1D:
    x = IntegerValue()

In [3]:
p1, p2 = Point1D(), Point1D()

In [4]:
p1.x = 10.1
p2.x = 20.2

In [5]:
p1.x, p2.x

(10, 20)

Как видите, теперь у нас есть дескриптор, который использует сами экземпляры для хранения данных:

In [6]:
p1.__dict__, p2.__dict__

({'stored_value': 10}, {'stored_value': 20})

Но вы заметите, что наш дескриптор жестко запрограммирован на использование того же ключа в словарях экземпляров, что приводит нас к этой проблеме:

In [7]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [8]:
p = Point2D()

In [9]:
p.x = 10.1

In [10]:
p.__dict__

{'stored_value': 10}

А что будет, если мы установим `y`? Какой символ будет использовать дескриптор для сохранения значения в экземпляре?

In [11]:
p.y = 20.2

In [12]:
p.__dict__

{'stored_value': 20}

Да, **тот самый** символ!

In [13]:
p.x, p.y

(20, 20)

Так что этот подход тоже не сработает. Каким-то образом нам нужно будет иметь отдельное имя хранилища для каждого свойства.

Мы могли бы сделать это, используя `__init__` нашего дескриптора:

In [14]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = '_' + name

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return getattr(instance, self._storage_name, None)

class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

In [15]:
p1 = Point2D()
p2 = Point2D()

In [16]:
p1.x = 10.1
p1.y = 20.2

In [17]:
p1.__dict__

{'_x': 10, '_y': 20}

In [18]:
p2.x = 100.1
p2.y = 200.2

In [19]:
p2.__dict__

{'_x': 100, '_y': 200}

Итак, этот подход может работать просто отлично, но есть несколько недостатков:

1. Пользователю нужно указать имя свойства дважды
2. Мы предполагаем, что `_` + `name` также не используется классом, в котором существует дескриптор (так что это может быть серьезной проблемой)
3. Мы предполагаем, что можем добавить атрибут к экземпляру — но что, если он использует слоты?

Один из способов обойти каждую из этих проблем — использовать сам экземпляр дескриптора для хранения значений экземпляра. Но, как мы видели ранее, мы не можем просто задать атрибут в экземпляре дескриптора, поскольку он будет общим для нескольких экземпляров класса, содержащего дескриптор.

Вместо этого мы собираемся **предположить**, что `instance` — это хешируемый объект, и использовать словарь в дескрипторе для хранения значений, специфичных для экземпляра:

In [20]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [21]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [22]:
p1 = Point2D()
p2 = Point2D()

In [23]:
p1.x = 10.1
p1.y = 20.2

In [24]:
p1.x, p1.y

(10, 20)

Фактически, мы можем видеть словарь в экземплярах дескриптора:

In [25]:
Point2D.x.values

{<__main__.Point2D at 0x7fa8e8204828>: 10}

In [26]:
Point2D.x.values

{<__main__.Point2D at 0x7fa8e8204828>: 10}

где ключом в обоих случаях является наш объект `p1`:

In [27]:
hex(id(p1))

'0x7fa8e8204828'

Теперь мы можем создать вторую точку и выполнить те же шаги:

In [28]:
p2 = Point2D()
p2.x = 100.1
p2.y = 200.2

In [29]:
hex(id(p2))

'0x7fa8b801bb00'

In [30]:
Point2D.x.values

{<__main__.Point2D at 0x7fa8e8204828>: 10,
 <__main__.Point2D at 0x7fa8b801bb00>: 100}

In [31]:
Point2D.y.values

{<__main__.Point2D at 0x7fa8e8204828>: 20,
 <__main__.Point2D at 0x7fa8b801bb00>: 200}

И все работает просто отлично:

In [32]:
p1.x, p1.y, p2.x, p2.y

(10, 20, 100, 200)

Или это так?

На самом деле у нас есть потенциальная утечка памяти — обратите внимание, что словарь в экземпляре дескриптора **также** хранит ссылку на объект точки — как **ключ** в словаре.

Давайте напишем простую вспомогательную функцию, которая позволяет нам получить количество ссылок для объекта по его идентификатору (и это имеет смысл только в том случае, если используемый нами идентификатор все еще имеет действительный неуничтоженный объект):

In [33]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [34]:
p1 = Point2D()
id_p1 = id(p1)

In [35]:
ref_count(id_p1)

1

Теперь давайте установим свойство `x` для `p1`:

In [36]:
p1.x = 100.1

Давайте еще раз проверим количество ссылок:

In [37]:
ref_count(id_p1)

2

Как вы видите, теперь это `2`. Если мы удалим нашу основную ссылку на `p1`, которая находится в нашем глобальном пространстве имен:

In [38]:
'p1' in globals()

True

In [39]:
del p1

In [40]:
'p1' in globals()

False

In [41]:
ref_count(id_p1)

1

И наш счетчик ссылок по-прежнему равен `1`, что означает, что сам объект не был уничтожен!

Фактически, мы можем видеть ссылку на этот объект в нашем словаре дескрипторов данных:

In [42]:
Point2D.x.values.items()

dict_items([(<__main__.Point2D object at 0x7fa8e8204828>, 10), (<__main__.Point2D object at 0x7fa8b801bb00>, 100), (<__main__.Point2D object at 0x7fa8e820a550>, 100)])

In [43]:
hex(id_p1)

'0x7fa8e820a550'

Как видите, ключ последнего элемента — это тот же идентификатор, на который ссылался `p1`.

Таким образом, хотя мы удалили `p1`, объект не был уничтожен — это может привести к утечке памяти.

Есть несколько способов справиться с этой проблемой. Первый, который мы рассмотрим, это то, что называется **слабыми ссылками**. Так что давайте перейдем к этому далее.

---